In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import datetime as dt
from technicals.indicators import RSI, MACD, ATR, BollingerBands ,heikin_ashi ,identify_pin_bar,EMA,ADX
from technicals.patterns import apply_patterns
from simulation.guru_tester import GuruTester
from models.trade_settings import TradeSettings
from constants import defs
import json
import pytz
pd.set_option('display.max_rows', None)  # Show all rows
# pd.set_option('display.max_columns', None)  # Show all columns


In [3]:
def apply_signal(row):
    """Modified to get all parameters from row"""
    # Trend Configuration
    primary_trend_up = row['EMA_20'] > row['EMA_50']
    secondary_trend_up = row['EMA_20'] > row['EMA_200']
    trend_strength = row['ADX_14'] > 25
    
    # Parameter Extraction
    maxspread = row['maxspread']
    mingain = row['mingain']
    atr_period = row['atr_period']
    rsi_period = row['rsi_period']
    rsi_oversold = row['rsi_oversold']
    rsi_overbought = row['rsi_overbought']
    
    # Volatility Check
    normalized_volatility = row[f"ATR_{atr_period}"] / row['mid_c']
    
    # Pattern Recognition
    bullish_pattern = (row['ENGULFING'] and row['PIN_BAR_BULL']) or row['MORNING_STAR']
    bearish_pattern = (row['ENGULFING'] and row['PIN_BAR_BEAR']) or row['EVENING_STAR']
    
    # Momentum Analysis
    rsi_ok = (row[f"RSI_{rsi_period}"] < rsi_oversold) if primary_trend_up else \
             (row[f"RSI_{rsi_period}"] > rsi_overbought)
    
    macd_ok = (row['MACD'] > row['SIGNAL_MD']) if primary_trend_up else (row['MACD'] < row['SIGNAL_MD'])
    
    # Entry Conditions
    long_conditions = all([
        row['SPREAD'] <= maxspread,
        row['GAIN'] >= mingain,
        trend_strength,
        primary_trend_up,
        secondary_trend_up,
        bullish_pattern,
        rsi_ok,
        macd_ok,
        row['session_ny_london'],
        normalized_volatility < 0.002,
        row['mid_c'] > row['EMA_20'],
        row['HA_Close'] > row['HA_Open']
    ])
    
    short_conditions = all([
        row['SPREAD'] <= maxspread,
        row['GAIN'] >= mingain,
        trend_strength,
        not primary_trend_up,
        not secondary_trend_up,
        bearish_pattern,
        rsi_ok,
        macd_ok,
        row['session_ny_london'],
        normalized_volatility < 0.002,
        row['mid_c'] < row['EMA_20'],
        row['HA_Close'] < row['HA_Open']
    ])
    
    return defs.BUY if long_conditions else defs.SELL if short_conditions else defs.NONE


In [4]:
def apply_SL(row):
    """Modified SL calculation using row parameters"""
    atr_multiplier = row['atr_multiplier']
    atr_period = row['atr_period']
    base_atr = row[f"ATR_{atr_period}"]
    
    if row['SIGNAL'] == defs.BUY:
        initial_sl = row['mid_c'] - (base_atr * 1.5)
        trailing_sl = row['mid_h'] - (base_atr * 0.5)
        return max(initial_sl, trailing_sl)
    elif row['SIGNAL'] == defs.SELL:
        initial_sl = row['mid_c'] + (base_atr * 1.5)
        trailing_sl = row['mid_l'] + (base_atr * 0.5)
        return min(initial_sl, trailing_sl)
    return 0.0

def apply_TP(row):
    """Modified TP calculation using row parameters"""
    riskreward = row['riskreward']
    atr_period = row['atr_period']
    base_atr = row[f"ATR_{atr_period}"]
    
    if row['SIGNAL'] == defs.BUY:
        recent_high = row['mid_h'].rolling(10).max()
        return recent_high + (base_atr * 0.5)
    elif row['SIGNAL'] == defs.SELL:
        recent_low = row['mid_l'].rolling(10).min()
        return recent_low - (base_atr * 0.5)
    return 0.0


In [5]:
df_an = pd.read_pickle(f"../data/EUR_USD_H1.pkl")
# print(df_an.head())


In [ ]:

def run_pair(pair):
        df_an = pd.read_pickle(f"../data/{pair}_H4.pkl")
        df_m5 = pd.read_pickle(f"../data/{pair}_M15.pkl")
        df_an.reset_index(drop=True, inplace=True)
        df_m5.reset_index(drop=True, inplace=True)
        with open("../bot/settings.json", "r") as f:
                data = json.loads(f.read())
                trade_settings = {k: TradeSettings(v, k) for k, v in data['pairs'].items()}
                trade_risk = data['trade_risk']
        pair_settings = trade_settings[pair]
        # print(pair_settings)

        df_an['time'] = pd.to_datetime(df_an['time'], utc=True)
        eastern_tz = pytz.timezone('US/Eastern')
        df_an['time'] = df_an['time'].dt.tz_convert(eastern_tz)
        df_an['hour'] = df_an['time'].dt.hour
        df_an['session_ny_london'] = df_an['hour'].between(8, 11)  # 8-11 AM EST
        df_an['time'] = df_an['time'].dt.strftime('%I:%M %p')
        # print(df_an.head(20))


        df_an['PAIR'] = pair
        df_an['SPREAD'] = df_an.ask_c - df_an.bid_c
        # print(df_an.head(20))


        df_an = RSI(df_an)
        df_an = BollingerBands(df_an, pair_settings.n_ma, pair_settings.n_std)
        df_an = ATR(df_an, pair_settings.atr_period)
        df_an = RSI(df_an, pair_settings.rsi_period)
        df_an = MACD(df_an)
        # df_an = momentum_reversal(df_an)
        # df_an = moving_average_crossover(df_an)
        df_an = heikin_ashi(df_an)
        df_an['EMA_20'] = EMA(df_an, 20)
        df_an['EMA_50'] = EMA(df_an, 50)
        df_an['EMA_200'] = EMA(df_an, 200)
        df_an['ADX_14'] = ADX(df_an)
        
       
        df_an =identify_pin_bar(df_an)
   
        
        df_an = apply_patterns(df_an)
        # print(df_an.head(20))
        df_an['EMA_200'] = df_an.mid_c.ewm(span=200, min_periods=200).mean()
        df_an['GAIN'] = abs(df_an.mid_c - df_an.BB_MA)
       
     
        df_an['TRADE_RISK'] = trade_risk
        df_an['n_ma'] = pair_settings.n_ma
        df_an['n_std'] = pair_settings.n_std
        df_an['maxspread']= pair_settings.maxspread
        df_an['mingain']= pair_settings.mingain
        df_an['riskreward']= pair_settings.riskreward
        df_an['atr_period'] = pair_settings.atr_period
        df_an['atr_multiplier']= pair_settings.atr_multiplier
        # df_an['atr_threshold']= pair_settings.atr_threshold
        df_an['rsi_period']= pair_settings.rsi_period
        df_an['rsi_overbought']= pair_settings.rsi_overbought
        df_an['rsi_oversold']= pair_settings.rsi_oversold
        df_an['SIGNAL'] = df_an.apply(apply_signal, axis=1)
        df_an['SL'] = df_an.apply(apply_SL, axis=1)
        df_an['TP'] = df_an.apply(apply_TP, axis=1)
        df_an['LOSS'] = abs(df_an.mid_c - df_an.SL)
        print(df_an.head(20))
        df_an['average_atr'] = df_an["atr_period"].rolling(50).mean()

        our_cols = [
        'PAIR', 'time', 'mid_c', 'EMA_20', 'EMA_50', 'EMA_200', 'ADX_14',
        'RSI_14', 'MACD', 'SIGNAL_MD', 'HA_Close', 'HA_Open',
        'ENGULFING', 'PIN_BAR_BULL', 'PIN_BAR_BEAR', 'SL', 'TP', 'SPREAD', 
        'GAIN', 'LOSS', 'SIGNAL','maxspread'
    ]


        df_slim = df_an[our_cols].copy()
        df_slim.dropna(inplace=True)

        df_slim.reset_index(drop=True, inplace=True)
        # print(df_slim.head())

        gt = GuruTester(
        df_slim,
        apply_signal,  
        df_m5,
        use_spread=True
        )

        gt.run_test()
        # return gt.df_results
        return gt.df_results.result.sum()




    

In [18]:
run_pair("EUR_USD")

        time  volume    mid_o    mid_h    mid_l    mid_c    bid_o    bid_h  \
0   05:00 PM    4868  1.05743  1.05806  1.05687  1.05735  1.05709  1.05799   
1   09:00 PM    8027  1.05732  1.05766  1.05660  1.05672  1.05725  1.05758   
2   01:00 AM   11230  1.05672  1.05776  1.05571  1.05583  1.05665  1.05768   
3   05:00 AM   17876  1.05583  1.05628  1.05369  1.05386  1.05576  1.05621   
4   09:00 AM   28535  1.05385  1.05710  1.05214  1.05338  1.05377  1.05702   
5   01:00 PM   34099  1.05339  1.05759  1.05166  1.05690  1.05332  1.05752   
6   05:00 PM    6414  1.05704  1.06021  1.05670  1.06020  1.05654  1.06014   
7   09:00 PM    7694  1.06019  1.06023  1.05933  1.05962  1.06011  1.06016   
8   01:00 AM   13942  1.05960  1.06102  1.05912  1.06004  1.05953  1.06095   
9   05:00 AM   21859  1.06008  1.06678  1.05990  1.06644  1.06000  1.06670   
10  09:00 AM   27002  1.06640  1.06670  1.06106  1.06155  1.06633  1.06663   
11  01:00 PM   10667  1.06156  1.06326  1.06140  1.06224  1.0614

KeyError: 'maxspread'

In [8]:
# res = []
# for p in [ "EUR_USD", "GBP_JPY" ]:
#     res.append(dict(pair=p,res=run_pair(p)))

In [9]:
# for r in res:
#     print(r['pair'], r['res'].result.sum())